# **Hi to all! This is my quick start notebook.**

# Hmm, the result of this notebook on the public dataset is interesting.... 0.540 is a lot, considering the failure of the model on the validation set, we will improve the model

# version 2 and 3 - Added cross validation

# version 4 - Added LGBM from this notebook https://www.kaggle.com/code/davidjlochner/base-tfidf-lgbm

# versions 5-8 - I tried changing the ngram parameter, but ngram=(1,1) seems to work best

# version 9 - Used autogluon to build the model

# version 12 - Make custom scorer fot the autogluon - This only worsened the result

# version 14-16 - Added cross validation 

# version 17 - Used new preprocessing from this great notebook https://www.kaggle.com/code/ye11725/tfidf-lgbm-baseline-with-code-comments

# version 21 - Used new model from this great notebook https://www.kaggle.com/code/ye11725/tfidf-lgbm-baseline-with-code-comments, which works much better

# version 23 - I will try to use GaussianNB to transform the results of the regression model into class labels

# version 25 - Restored the previous version because the idea didn't work

# version 27 - The number of folds has been increased

# version 31 - add pseudo labeling: The idea did not work

# version 35 - Added new post-processing and result analysisk

# version 36 - Added CountVectorizer from https://www.kaggle.com/code/hideyukizushi/aes2-5folddeberta-lgbm-countvectorizer-lb-810

# version 44 - Partially added pipeline from here https://www.kaggle.com/code/hideyukizushi/aes2-deberta-lgbm-countvectorizer-lb-814?scriptVersionId=173181256

# version 51 - Added feature selection from here https://www.kaggle.com/code/yongsukprasertsuk/0-818-deberta-v3-large-lgbm-baseline

Choose the appropriate version of autogluon to train on GPU or CPU

In [ ]:
#!pip install -q --no-index --find-links=/kaggle/input/autogluon-cpu autogluon
#!pip install -q --no-index --find-links=/kaggle/input/autogluon-gpu autogluon

In [ ]:
# Importing necessary libraries
import torch
import copy
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments,DataCollatorWithPadding
import nltk
from datasets import Dataset
from glob import glob
import numpy as np 
import pandas as pd
import polars as pl
import re
import random
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import gc
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import nltk
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import random
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier,GradientBoostingClassifier,BaggingClassifier
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB,MultinomialNB,ComplementNB
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
#from autogluon.tabular.models import NNFastAiTabularModel
#from autogluon.tabular import TabularDataset, TabularPredictor
from lightgbm import log_evaluation, early_stopping
from sklearn.linear_model import SGDClassifier
import polars as pl
import pickle
from scipy.special import softmax
nltk.download('wordnet')

In [ ]:
MAX_LENGTH = 1024
TEST_DATA_PATH = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv"
MODEL_PATH = '/kaggle/input/aes2-400-20240419134941/*/*'
EVAL_BATCH_SIZE = 1
models = glob(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(models[0])

def tokenize(sample):
    return tokenizer(sample['full_text'], max_length=MAX_LENGTH, truncation=True)

df_test = pd.read_csv(TEST_DATA_PATH)
ds = Dataset.from_pandas(df_test).map(tokenize).remove_columns(['essay_id', 'full_text'])

args = TrainingArguments(
    ".", 
    per_device_eval_batch_size=EVAL_BATCH_SIZE, 
    report_to="none"
)

predictions = []
for model in models:
    model = AutoModelForSequenceClassification.from_pretrained(model)
    trainer = Trainer(
        model=model, 
        args=args, 
        data_collator=DataCollatorWithPadding(tokenizer), 
        tokenizer=tokenizer
    )    
    preds = trainer.predict(ds).predictions
    predictions.append(softmax(preds, axis=-1))
    del model, trainer
    torch.cuda.empty_cache()
    gc.collect()
    
predicted_score = 0.
for p in predictions:
    predicted_score += p
    
predicted_score /= len(predictions)
df_test['score'] = predicted_score.argmax(-1) + 1
df_test.head()
df_test[['essay_id', 'score']].to_csv('submission1.csv', index=False)

In [ ]:
columns = [  
    (
        pl.col("full_text").str.split(by="\n\n").alias("paragraph")
    ),
]
PATH = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/"
# 载入训练集和测试集，同时对full_text数据使用\n\n字符分割为列表，重命名为paragraph
# Load training and testing sets, while using \ n \ n character segmentation to list and renaming to paragraph for full_text data
train = pl.read_csv(PATH + "train.csv").with_columns(columns)
test = pl.read_csv(PATH + "test.csv").with_columns(columns)
# 显示训练集中的第一个样本数据
# Display the first sample data in the training set
train.head(1)

In [ ]:
# Importing the ProfileReport class from the ydata_profiling library
#from ydata_profiling import ProfileReport

# Generating a profile report for the train dataset
# Setting the title of the report to "Pandas Profiling Report"
#profile = ProfileReport(train, title="Pandas Profiling Report")

# Displaying the generated profile report
#profile

**A quick analysis showed that the class distribution resembles a normal distribution, interestingly….**


**Let's try to build a simple base model so we have something to go off of.
I will use a bag-of-words model with the addition of feature selection, which can simplify the classification model.**


# **Preprocessing**

## Features engineering - https://www.kaggle.com/code/ye11725/tfidf-lgbm-baseline-with-code-comments

In [ ]:
cList = {
  "ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because",  "could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not",
  "haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will","he'll've": "he will have","he's": "he is",
  "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is","I'd": "I would","I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have",
  "isn't": "is not","it'd": "it had","it'd've": "it would have","it'll": "it will", "it'll've": "it will have","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not",
  "might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have","needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not","oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
  "shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
  "should've": "should have","shouldn't": "should not","shouldn't've": "should not have","so've": "so have","so's": "so is","that'd": "that would","that'd've": "that would have","that's": "that is","there'd": "there had","there'd've": "there would have","there's": "there is","they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we had",
  "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
  "weren't": "were not","what'll": "what will","what'll've": "what will have",
  "what're": "what are","what's": "what is","what've": "what have","when's": "when is","when've": "when have",
  "where'd": "where did","where's": "where is","where've": "where have","who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is",
  "why've": "why have","will've": "will have","won't": "will not","won't've": "will not have","would've": "would have","wouldn't": "would not",
  "wouldn't've": "would not have","y'all": "you all","y'alls": "you alls","y'all'd": "you all would",
  "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have","you're": "you are",  "you've": "you have"
   }

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
def dataPreprocessing(x):
    # Convert words to lowercase
    x = x.lower()
    # Remove HTML
    x = removeHTML(x)
    # Delete strings starting with @
    x = re.sub("@\w+", '',x)
    # Delete Numbers
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    # Delete URL
    x = re.sub("http\w+", '',x)
    # Replace consecutive empty spaces with a single space character
    x = re.sub(r"\s+", " ", x)
    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
    # Remove empty characters at the beginning and end
    x = x.strip()
    return x

### 2.Paragraph Features

In [ ]:
# 段落特征
# paragraph features
def Paragraph_Preprocess(tmp):
    # 将段落列表扩展为一行行的数据
    # Expand the paragraph list into several lines of data
    tmp = tmp.explode('paragraph')
    # 段落预处理
    # Paragraph preprocessing
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(dataPreprocessing))
    # 计算每一个段落的长度
    # Calculate the length of each paragraph
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x)).alias("paragraph_len"))
    # 计算每一个段落中句子的数量和单词的数量
    # Calculate the number of sentences and words in each paragraph
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x.split('.'))).alias("paragraph_sentence_cnt"),
                    pl.col('paragraph').map_elements(lambda x: len(x.split(' '))).alias("paragraph_word_cnt"),)
    return tmp
# feature_eng
paragraph_fea = ['paragraph_len','paragraph_sentence_cnt','paragraph_word_cnt']
def Paragraph_Eng(train_tmp):
    aggs = [
        # 统计段落长度大于和小于 i 值的个数
        # Count the number of paragraph lengths greater than and less than the i-value
        *[pl.col('paragraph').filter(pl.col('paragraph_len') >= i).count().alias(f"paragraph_{i}_cnt") for i in [50,75,100,125,150,175,200,250,300,350,400,500,600,700] ], 
        *[pl.col('paragraph').filter(pl.col('paragraph_len') <= i).count().alias(f"paragraph_{i}_cnt") for i in [25,49]], 
        # 其他
        # other
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in paragraph_fea],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in paragraph_fea],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in paragraph_fea],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in paragraph_fea],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in paragraph_fea],

        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df
tmp = Paragraph_Preprocess(train)
train_feats = Paragraph_Eng(tmp)
train_feats['score'] = train['score']
# 获取特征名称
# Obtain feature names
feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

### 3.Sentence Features

In [ ]:
# sentence feature
def Sentence_Preprocess(tmp):
    # 对full_text预处理，并且使用句号分割出文本的句子
    # Preprocess full_text and use periods to segment sentences in the text
    tmp = tmp.with_columns(pl.col('full_text').map_elements(dataPreprocessing).str.split(by=".").alias("sentence"))
    tmp = tmp.explode('sentence')
    # 计算句子的长度
    # Calculate the length of a sentence
    tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x)).alias("sentence_len"))
    # 筛选出句子长度大于15的那一部分数据
    # Filter out the portion of data with a sentence length greater than 15
    tmp = tmp.filter(pl.col('sentence_len')>=15)
    # 统计每一句中单词的数量
    # Count the number of words in each sentence
    tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x.split(' '))).alias("sentence_word_cnt"))
    
    return tmp
# feature_eng
sentence_fea = ['sentence_len','sentence_word_cnt']
def Sentence_Eng(train_tmp):
    aggs = [
        # 统计句子长度大于 i 的句子个数
        # Count the number of sentences with a length greater than i
        *[pl.col('sentence').filter(pl.col('sentence_len') >= i).count().alias(f"sentence_{i}_cnt") for i in [15,50,100,150,200,250,300] ], 
        # 其他
        # other
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in sentence_fea],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in sentence_fea],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in sentence_fea],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in sentence_fea],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in sentence_fea],

        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

tmp = Sentence_Preprocess(train)
# 将新生成的特征数据和之前生成的特征数据合并
# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(Sentence_Eng(tmp), on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

### 4.Word Features

In [ ]:
# word feature
def Word_Preprocess(tmp):
    # 对full_text预处理，并且使用空格符分割出文本的单词
    # Preprocess full_text and use spaces to separate words from the text
    tmp = tmp.with_columns(pl.col('full_text').map_elements(dataPreprocessing).str.split(by=" ").alias("word"))
    tmp = tmp.explode('word')
    # 计算每一个的单词长度
    # Calculate the length of each word
    tmp = tmp.with_columns(pl.col('word').map_elements(lambda x: len(x)).alias("word_len"))
    # 删除单词长度为0的数据
    # Delete data with a word length of 0
    tmp = tmp.filter(pl.col('word_len')!=0)
    
    return tmp
# feature_eng
def Word_Eng(train_tmp):
    aggs = [
        # 统计单词长度大于 i+1 的单词个数
        # Count the number of words with a length greater than i+1
        *[pl.col('word').filter(pl.col('word_len') >= i+1).count().alias(f"word_{i+1}_cnt") for i in range(15) ], 
        # 其他
        # other
        pl.col('word_len').max().alias(f"word_len_max"),
        pl.col('word_len').mean().alias(f"word_len_mean"),
        pl.col('word_len').std().alias(f"word_len_std"),
        pl.col('word_len').quantile(0.25).alias(f"word_len_q1"),
        pl.col('word_len').quantile(0.50).alias(f"word_len_q2"),
        pl.col('word_len').quantile(0.75).alias(f"word_len_q3"),
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

tmp = Word_Preprocess(train)
# 将新生成的特征数据和之前生成的特征数据合并
# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(Word_Eng(tmp), on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

### 5.Tf-idf features

In [ ]:
# TfidfVectorizer parameter
vectorizer = TfidfVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(4,8),
            min_df=0.05,
            max_df=0.95,
            sublinear_tf=True,
)
# 将全部数据集都填充进TfidfVectorizer里，这可能会造成泄露和过于乐观的CV分数
# Fit all datasets into TfidfVector,this may cause leakage and overly optimistic CV scores
train_tfid = vectorizer.fit_transform([i for i in train['full_text']])
# 转换为数组
# Convert to array
dense_matrix = train_tfid.toarray()
# 转换为dataframe
# Convert to dataframe
df = pd.DataFrame(dense_matrix)
# 重命名特征
# rename features
tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = train_feats['essay_id']
# 将新生成的特征数据和之前生成的特征数据合并
# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(df, on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

### 6.CountVectorizer

In [ ]:
vectorizer_cnt = CountVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(3,5),
            min_df=0.10,
            max_df=0.85,
)
train_tfid = vectorizer_cnt.fit_transform([i for i in train['full_text']])
dense_matrix = train_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_cnt_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = train_feats['essay_id']
train_feats = train_feats.merge(df, on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

### 7. Meta Features(deberta-v3-large)

In [ ]:
import joblib

deberta_oof = joblib.load('/kaggle/input/aes2-400-20240419134941/oof.pkl')
print(deberta_oof.shape, train_feats.shape)

for i in range(6):
    train_feats[f'deberta_oof_{i}'] = deberta_oof[:, i]

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))    

train_feats.shape

# **Model training**

In [ ]:
# idea from https://www.kaggle.com/code/rsakata/optimize-qwk-by-lgb/notebook#QWK-objective
def quadratic_weighted_kappa(y_true, y_pred):
    y_true = y_true + a
    y_pred = (y_pred + a).clip(1, 6).round()
    qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")
    return 'QWK', qwk, True
def qwk_obj(y_true, y_pred):
    labels = y_true + a
    preds = y_pred + a
    preds = preds.clip(1, 6)
    f = 1/2*np.sum((preds-labels)**2)
    g = 1/2*np.sum((preds-a)**2+b)
    df = preds - labels
    dg = preds - a
    grad = (df/g - f*dg/g**2)*len(labels)
    hess = np.ones(len(labels))
    return grad, hess
a = 2.948
b = 1.092


In [ ]:
# Converting the 'text' column to string type and assigning to X
X = train_feats[feature_names].astype(np.float32).values

# Converting the 'score' column to integer type and assigning to y
y_split = train_feats['score'].astype(int).values
y = train_feats['score'].astype(np.float32).values-a
oof = train_feats['score'].astype(np.float32).values


# Feature Selection¶(https://www.kaggle.com/code/yongsukprasertsuk/0-818-deberta-v3-large-lgbm-baseline,  https://www.kaggle.com/code/xianhellg/more-feature-engineering-feature-selection-0-817)


In [ ]:
if_train=True # You can train your model here, I'm currently using the models trained in version 51 of this notebook


In [ ]:

if if_train:
    # Define the number of splits for cross-validation
    n_splits = 15
    fse=[]
    # Initialize StratifiedKFold with the specified number of splits
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)

    # Lists to store scores
    f1_scores = []
    kappa_scores = []
    models = []
    predictions = []
    callbacks = [log_evaluation(period=25), early_stopping(stopping_rounds=75,first_metric_only=True)]
    # Loop through each fold of the cross-validation
    i=1
    for train_index, test_index in skf.split(X, y_split):
        # Split the data into training and testing sets for this fold
        print('fold',i)
        X_train_fold, X_test_fold = X[train_index], X[test_index]


        y_train_fold, y_test_fold, y_test_fold_int = y[train_index], y[test_index], y_split[test_index]



        model = lgb.LGBMRegressor(
                    objective = qwk_obj,
                    metrics = 'None',
                    learning_rate = 0.1,
                    max_depth = 5,
                    num_leaves = 10,
                    colsample_bytree=0.5,
                    reg_alpha = 0.1,
                    reg_lambda = 0.8,
                    n_estimators=1024,
                    random_state=42,
                    extra_trees=True,
                    class_weight='balanced',

                    verbosity = - 1)

        # Fit the model on the training data for this fold

        predictor = model.fit(X_train_fold,
                                      y_train_fold,
                                      eval_names=['train', 'valid'],
                                      eval_set=[(X_train_fold, y_train_fold), (X_test_fold, y_test_fold)],
                                      eval_metric=quadratic_weighted_kappa,
                                      callbacks=callbacks,)
        models.append(predictor)
        # Make predictions on the test data for this fold
        predictions_fold = predictor.predict(X_test_fold)
        predictions_fold = predictions_fold + a
        predictions_fold = predictions_fold.clip(1, 6).round()
        predictions.append(predictions_fold)
        # Calculate and store the F1 score for this fold
        f1_fold = f1_score(y_test_fold_int, predictions_fold, average='weighted')
        f1_scores.append(f1_fold)

        # Calculate and store the Cohen's kappa score for this fold
        kappa_fold = cohen_kappa_score(y_test_fold_int, predictions_fold, weights='quadratic')
        kappa_scores.append(kappa_fold)

        # Calculating the confusion matrix
        cm = confusion_matrix(y_test_fold_int, predictions_fold, labels=[x for x in range(1,7)])

        # Displaying the confusion matrix
        disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                      display_labels=[x for x in range(1,7)])
        disp.plot()
        plt.show()
        print(f'F1 score across fold: {f1_fold}')
        print(f'Cohen kappa score across fold: {kappa_fold}')
        i+=1
        fse.append(pd.Series(predictor.feature_importances_, feature_names))
    # Calculate the mean scores across all folds
    mean_f1_score = np.mean(f1_scores)
    mean_kappa_score = np.mean(kappa_scores)

    # Print the mean scores
    print(f'Mean F1 score across {n_splits} folds: {mean_f1_score}')
    print(f'Mean Cohen kappa score across {n_splits} folds: {mean_kappa_score}')

    # Зберегти список моделей у файл
    with open('models_feature_select.pkl', 'wb') as f:
        pickle.dump(models, f)
    with open('fse.pkl', 'wb') as f:
        pickle.dump(fse, f)
else:
    with open('/kaggle/input/feature-select-learning-agency-lab/models_feature_select.pkl', 'rb') as f:
        models = pickle.load(f)
    with open('/kaggle/input/feature-select-learning-agency-lab/fse.pkl', 'rb') as f:
        fse = pickle.load(f)
    

# **Let's use cross-validation**

In [ ]:
if_train=True # You can train your model here, I'm currently using the models trained in version 51 of this notebook


In [ ]:

if if_train:
    # Define the number of splits for cross-validation
    n_splits = 15

    # Initialize StratifiedKFold with the specified number of splits
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)

    # Lists to store scores
    f1_scores = []
    kappa_scores = []
    models = []
    predictions = []
    callbacks = [log_evaluation(period=25), early_stopping(stopping_rounds=75,first_metric_only=True)]
    # Loop through each fold of the cross-validation
    i=1
    for train_index, test_index in skf.split(X, y_split):
        # Split the data into training and testing sets for this fold
        print('fold',i)
        feature_select = fse[i-1].sort_values(ascending=False).index.tolist()[:13000]
        X_train_fold, X_test_fold = train_feats[feature_select].astype(np.float32).values[train_index], train_feats[feature_select].astype(np.float32).values[test_index]

        
        y_train_fold, y_test_fold, y_test_fold_int = y[train_index], y[test_index], y_split[test_index]



        model = lgb.LGBMRegressor(
                    objective = qwk_obj,
                    metrics = 'None',
                    learning_rate = 0.1,
                    max_depth = 5,
                    num_leaves = 10,
                    colsample_bytree=0.5,
                    reg_alpha = 0.1,
                    reg_lambda = 0.8,
                    n_estimators=1024,
                    random_state=42,
                    extra_trees=True,
                    class_weight='balanced',

                    verbosity = - 1)

        # Fit the model on the training data for this fold

        predictor = model.fit(X_train_fold,
                                      y_train_fold,
                                      eval_names=['train', 'valid'],
                                      eval_set=[(X_train_fold, y_train_fold), (X_test_fold, y_test_fold)],
                                      eval_metric=quadratic_weighted_kappa,
                                      callbacks=callbacks,)
        models.append(predictor)
        # Make predictions on the test data for this fold
        predictions_fold = predictor.predict(X_test_fold)
        predictions_fold = predictions_fold + a
        predictions_fold = predictions_fold.clip(1, 6).round()
        predictions.append(predictions_fold)
        # Calculate and store the F1 score for this fold
        f1_fold = f1_score(y_test_fold_int, predictions_fold, average='weighted')
        f1_scores.append(f1_fold)

        # Calculate and store the Cohen's kappa score for this fold
        kappa_fold = cohen_kappa_score(y_test_fold_int, predictions_fold, weights='quadratic')
        kappa_scores.append(kappa_fold)

        # Calculating the confusion matrix
        cm = confusion_matrix(y_test_fold_int, predictions_fold, labels=[x for x in range(1,7)])

        # Displaying the confusion matrix
        disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                      display_labels=[x for x in range(1,7)])
        disp.plot()
        plt.show()
        print(f'F1 score across fold: {f1_fold}')
        print(f'Cohen kappa score across fold: {kappa_fold}')
        i+=1
    # Calculate the mean scores across all folds
    mean_f1_score = np.mean(f1_scores)
    mean_kappa_score = np.mean(kappa_scores)

    # Print the mean scores
    print(f'Mean F1 score across {n_splits} folds: {mean_f1_score}')
    print(f'Mean Cohen kappa score across {n_splits} folds: {mean_kappa_score}')

    # Зберегти список моделей у файл
    with open('models.pkl', 'wb') as f:
        pickle.dump(models, f)
else:
    with open('/kaggle/input/quick-start-lgbm-version-30/models.pkl', 'rb') as f:
        models = pickle.load(f)
    

In [ ]:

# Define the number of splits for cross-validation
n_splits = 15

# Initialize StratifiedKFold with the specified number of splits
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
oof=oof.astype('float32')

# Loop through each fold of the cross-validation
i=0
for train_index, test_index in skf.split(X, y_split):
    # Split the data into training and testing sets for this fold
    print('fold',i)
    feature_select = fse[i].sort_values(ascending=False).index.tolist()[:13000]
    X_train_fold, X_test_fold = train_feats[feature_select].astype(np.float32).values[train_index], train_feats[feature_select].astype(np.float32).values[test_index]


    y_train_fold, y_test_fold, y_test_fold_int = y[train_index], y[test_index], y_split[test_index]



    predictor = models[i]
    # Make predictions on the test data for this fold
    predictions_fold = predictor.predict(X_test_fold)
    predictions_fold = predictions_fold + a
    oof[test_index]=predictions_fold
    #predictions_fold = predictions_fold.clip(1, 6).round()
    
    i+=1
oof

# Displaying the results of model predictions

In [ ]:
 # Calculate and store the F1 score for this fold
mean_f1_score = f1_score(y_split, oof.clip(1, 6).round(), average='weighted')

# Calculate and store the Cohen's kappa score for this fold
mean_kappa_score = cohen_kappa_score(y_split, oof.clip(1, 6).round(), weights='quadratic')

# Calculating the confusion matrix
cm = confusion_matrix(y_split, oof.clip(1, 6).round(), labels=[x for x in range(1,7)])

# Displaying the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=[x for x in range(1,7)])
disp.plot()
plt.show()
# Print the mean scores
print(f'Mean F1 score across {n_splits} folds: {mean_f1_score}')
print(f'Mean Cohen kappa score across {n_splits} folds: {mean_kappa_score}')

In [ ]:
import matplotlib.pyplot as plt

# Припустимо, що oof та y_split - це ваші масиви

# Створюємо кольорову палітру для різних класів
colors = ['r', 'g', 'b', 'c', 'm', 'y']

# Отримуємо унікальні класи з y_split
unique_classes = set(y_split)

# Створюємо підграфік для візуалізації
plt.figure()

# Проходимося по кожному класу і відображаємо відповідні точки з oof
for cls in unique_classes:
    # Відфільтруємо точки з oof, які відповідають поточному класу
    points = oof[y_split == cls]
    # Відображаємо точки для поточного класу з відповідним кольором
    plt.scatter(points, [cls] * len(points), color=colors[cls-1], label=f'Class {cls}')

# Налаштуємо заголовок та мітки осей
plt.title('Visualization of oof with different classes from y_split')
plt.xlabel('oof values')
plt.ylabel('Class')

# Додаємо легенду
plt.legend()

# Показуємо графік
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Припустимо, що oof та y_split - це ваші масиви

# Отримуємо унікальні класи з y_split
unique_classes = sorted(set(y_split))

# Знайдемо максимальне значення для побудови однакової сітки
max_value = max([max(oof[y_split == cls]) for cls in unique_classes])

# Створюємо новий графік для візуалізації
plt.figure(figsize=(20, 12))

# Проходимося по кожному класу і будуємо частотний графік для відповідних точок oof
for idx, cls in enumerate(unique_classes):
    # Відфільтруємо точки з oof, які відповідають поточному класу
    points = oof[y_split == cls]
    # Побудова частотного графіка для поточного класу
    plt.subplot(2, 3, idx+1)  # Побудова у вигляді 2x3 підграфіків
    plt.hist(points, color='skyblue', edgecolor='black', bins=20)
    plt.title(f'Class {cls}')
    plt.xlabel('oof values')
    plt.ylabel('Frequency')
    plt.xlim(0, max_value)  # Встановлення однакового діапазону для осі x
    plt.ylim(0, None)  # Встановлення діапазону для осі y (від 0 до максимума)

# Налаштуємо заголовок та підзаголовки
plt.suptitle('Frequency Distribution of oof for each y_split class')
plt.tight_layout()

# Показуємо графік
plt.show()


# Let's try to build a classification with thresholds

In [ ]:
import numpy as np

# Оптимальні пороги для переходу між класами
optimal_thresholds = [1.5, 2.5, 3.5, 4.5, 5.5]
oof_clip = oof.clip(1, 6)
def classify_with_thresholds(oof_clip, optimal_thresholds):

    # Класифікуємо значення oof за допомогою оптимальних порогів
    classified_oof = np.empty_like(oof_clip)
    for i, threshold in enumerate(optimal_thresholds):
        if i == 0:
            classified_oof[oof_clip < threshold] = 1

        else:
            classified_oof[(oof_clip >= optimal_thresholds[i-1]) & (oof_clip < threshold)] = i + 1

        if i == 4:
            classified_oof[oof_clip >= threshold] = 6
            
    return classified_oof.astype(int)
            
classified_oof =classify_with_thresholds(oof_clip, optimal_thresholds)
    
print("Classified oof:")
print(classified_oof)

In [ ]:
 # Calculate and store the F1 score for this fold
mean_f1_score = f1_score(y_split, classified_oof, average='weighted')

# Calculate and store the Cohen's kappa score for this fold
mean_kappa_score = cohen_kappa_score(y_split, classified_oof, weights='quadratic')

# Calculating the confusion matrix
cm = confusion_matrix(y_split, classified_oof, labels=[x for x in range(1,7)])

# Displaying the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=[x for x in range(1,7)])
disp.plot()
plt.show()
# Print the mean scores
print(f'Mean F1 score across {n_splits} folds: {mean_f1_score}')
print(f'Mean Cohen kappa score across {n_splits} folds: {mean_kappa_score}')

In [ ]:
optimal_thresholds = [1.5, 2.5, 3.5, 4.5, 5.5]
best_kappa_score=mean_kappa_score
for i in range(5):
    i_threshold=optimal_thresholds[i]-0.5
    for j in range(1000):
        thresholds=[x for x in optimal_thresholds]
        thresholds[i]=i_threshold+j/1000
        
        classified_oof =classify_with_thresholds(oof_clip, thresholds)
        threshold_kappa_score=cohen_kappa_score(y_split, classified_oof, weights='quadratic')
        #print(i,thresholds,threshold_kappa_score)
        if threshold_kappa_score>=best_kappa_score:
            best_kappa_score=threshold_kappa_score
            optimal_thresholds[i]=thresholds[i]
optimal_thresholds

In [ ]:
classified_oof =classify_with_thresholds(oof_clip, optimal_thresholds)
 # Calculate and store the F1 score for this fold
mean_f1_score = f1_score(y_split, classified_oof, average='weighted')

# Calculate and store the Cohen's kappa score for this fold
mean_kappa_score = cohen_kappa_score(y_split, classified_oof, weights='quadratic')

# Calculating the confusion matrix
cm = confusion_matrix(y_split, classified_oof, labels=[x for x in range(1,7)])

# Displaying the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=[x for x in range(1,7)])
disp.plot()
plt.show()
# Print the mean scores
print(f'Mean F1 score across {n_splits} folds: {mean_f1_score}')
print(f'Mean Cohen kappa score across {n_splits} folds: {mean_kappa_score}')

# **Inference**

In [ ]:
# Paragraph
tmp = Paragraph_Preprocess(test)
test_feats = Paragraph_Eng(tmp)
# Sentence
tmp = Sentence_Preprocess(test)
test_feats = test_feats.merge(Sentence_Eng(tmp), on='essay_id', how='left')
# Word
tmp = Word_Preprocess(test)
test_feats = test_feats.merge(Word_Eng(tmp), on='essay_id', how='left')

# Tfidf
test_tfid = vectorizer.transform([i for i in test['full_text']])
dense_matrix = test_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = test_feats['essay_id']
test_feats = test_feats.merge(df, on='essay_id', how='left')

# CountVectorizer
test_tfid = vectorizer_cnt.transform([i for i in test['full_text']])
dense_matrix = test_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_cnt_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = test_feats['essay_id']
test_feats = test_feats.merge(df, on='essay_id', how='left')

# Meta Features(deberta-v3-large)
for i in range(6):
    test_feats[f'deberta_oof_{i}'] = predicted_score[:, i]

# Features number
feature_names = list(filter(lambda x: x not in ['essay_id','score'], test_feats.columns))
print('Features number: ',len(feature_names))
test_feats.head(3)

In [ ]:
probabilities = []
for i in range(len(models)):
    feature_select = fse[i].sort_values(ascending=False).index.tolist()[:13000]
    proba= models[i].predict(test_feats[feature_select])+ a
    probabilities.append(proba)
# Compute the average probabilities across all models
predictions = np.mean(probabilities, axis=0)

#predictions = predictions.clip(1, 6)


predictions = np.round(predictions.clip(1, 6))
#predictions =classify_with_thresholds(predictions, optimal_thresholds)
# Print the predictions
print(predictions)


In [ ]:
#test_vector=new_pipeline.transform(test['text'])
#predictions=lgb_model.predict(test_vector)
#predictions

In [ ]:
#predictions = model.predict(test['text'])
#predictions
# Initialize an empty list to store probabilities for each model
"""
probabilities = []

# Iterate over each model in the list of models
for model in models:
    # Make predictions using the current model
    print(model.predict(test['text']))
    
    # Compute decision scores for the current model
    decision_scores = model.decision_function(test['text'])
    
    # Convert decision scores to probabilities using the sigmoid function
    proba = 1 / (1 + np.exp(-decision_scores))
    # Append the probabilities to the list
    probabilities.append(proba)

# Compute the average probabilities across all models
probabilities = np.mean(probabilities, axis=0)

# Determine the predicted class by selecting the class with the highest probability
# Add 1 to the index to match the class labels (assuming classes start from 1)
predictions = np.argmax(probabilities, axis=1) + 1

# Print the predictions
print(predictions)
"""

In [ ]:
submission=pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv")
submission['score']=predictions
submission['score']=submission['score'].astype(int)
submission.to_csv("submission.csv",index=None)
display(submission.head())